#### This IPython notebook explores the data for preprocessing. The final data preprocessing will be done in the script file "Data_Preprocess.py".

In [1]:
import numpy as np
import pandas as pd
from langdetect import detect_langs
import joblib

In [2]:
df = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\UbuntuDialogueText\UbuntuDialogueText.csv')

In [5]:
df.head(10)

,folder,dialogueID,date,from,to,text
0,3,126125.tsv,2008-04-23T14:55:00.000Z,bad_image,NaN,"Hello folks, please help me a bit with the fol..."
1,3,126125.tsv,2008-04-23T14:56:00.000Z,bad_image,NaN,Did I choose a bad channel? I ask because you ...
2,3,126125.tsv,2008-04-23T14:57:00.000Z,lordleemo,bad_image,the second sentence is better english and we...
3,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,Sock Puppe?t
4,3,64545.tsv,2009-08-01T06:22:00.000Z,mechtech,NaN,WTF?
5,3,64545.tsv,2009-08-01T06:22:00.000Z,richardcavell,mechtech,"it's a wikipedia term. There is Prodigy, PRDI..."
6,3,98758.tsv,2009-05-20T07:58:00.000Z,C-00000100,Severity1,what is bartek trying to do?
7,3,98758.tsv,2009-05-20T08:01:00.000Z,C-00000100,Severity1,is he trying to play online video?
8,3,98758.tsv,2009-05-20T08:04:00.000Z,C-00000100,Severity1,"bartek go here, if this doesn't play then it g..."
9,3,86303.tsv,2010-08-24T19:56:00.000Z,pozic,Somelauw,the algorithms from aptitude were ported to ap...


In [5]:
df.shape

(1038324, 6)

In [17]:
df['from'].unique()

array(['bad_image', 'lordleemo', 'mechtech', ..., 'torias', 'alexs',
       'lokpest_'], dtype=object)

In [18]:
df['from'].nunique()

164083

In [6]:
df['to'].unique()

array([nan, 'bad_image', 'mechtech', ..., 'torias', 'alexs', 'lokpest_'],
      dtype=object)

In [16]:
df['to'].nunique()

149488

In [19]:
df['dialogueID'].unique()

array(['126125.tsv', '64545.tsv', '98758.tsv', ..., '19585.tsv',
       '51506.tsv', '99669.tsv'], dtype=object)

In [20]:
df['dialogueID'].nunique()

346108

In [7]:
for i in range(10):
    print(df['text'][i])

Hello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?
Did I choose a bad channel? I ask because you seem to be dumb like windows user
the second sentence is better english   and we are not dumb
Sock Puppe?t
WTF?
it's a wikipedia term.  There is Prodigy, PRDIGY and prgidy and they're all the same guy
what is bartek trying to do?
is he trying to play online video?
bartek go here, if this doesn't play then it gives you a link to get it... http://www.milkandcookies.com/link/45466/detail/
the algorithms from aptitude were ported to apt-get, AFAIK.


In [2]:
gpt_chatlogs = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\ChatGPT_chatlogs.csv')

In [3]:
gpt_chatlogs.shape

(89311, 3)

In [10]:
gpt_chatlogs.tail(1)

,post_number,system_message,conversation
89310,[22628],[''],"[{'user': 'Anonymous', 'message': 'Can you exp..."


In [16]:
gpt_chatlogs['conversation'].iloc[-1]

'[{\'user\': \'Anonymous\', \'message\': \'Can you explain the SOLID principles in OOP?\'}, {\'user\': \'Chat GPT\', \'message\': \'SOLID is an acronym for five design principles in object-oriented programming (OOP). These principles aim to make software design more understandable, flexible, and maintainable. The SOLID principles are: Single Responsibility Principle (SRP): A class should have only one reason to change, and that reason should be encapsulated within the class. Open-Closed Principle (OCP): A class should be open for extension but closed for modification. Liskov Substitution Principle (LSP): Subtypes should be substitutable for their base types. Interface Segregation Principle (ISP): A class should not be forced to implement interfaces it does not use. Dependency Inversion Principle (DIP): High-level modules should not depend on low-level modules, but both should depend on abstractions. By following these principles, developers can create code that is more robust, easier t

In [5]:
import mailbox

def convert_mbox_to_txt(mbox_file, output_file):
    mbox = mailbox.mbox(mbox_file)  # Open the mbox file
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for message in mbox:
                if message.is_multipart():
                    for part in message.walk():
                        if part.get_content_type() == 'text/plain':
                            payload = part.get_payload(decode=True)
                            if payload:
                                f.write(payload.decode('utf-8'))
                                break
                else:
                    # Handle single part messages
                    if message.get_content_type() == 'text/plain':
                        payload = message.get_payload(decode=True)
                        if payload:
                            f.write(payload.decode('utf-8'))
    finally:
        mbox.close()  # Close the mbox file after processing

# Example usage
mbox_file = r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\mbox\Inbox.mbox'
output_file = r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\mbox\Inbox.txt'
convert_mbox_to_txt(mbox_file, output_file)

In [ ]:
ChatGPT_chatlogs = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\ChatGPT_chatlogs.csv', engine='python')

#### Preprocess the Chat GPT dialog dataset to question and answer dataset

In [1]:
import numpy as np
import pandas as pd
from langdetect import detect_langs
from tqdm import tqdm
import joblib

In [4]:
clg = pd.read_excel(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\ChatGPT_chatlogs_repaired.xlsx')

In [5]:
def pure_en(text):
    try:
        languages = detect_langs(text)
        for lang in languages:
            if lang.lang == 'en' and lang.prob > 0.99999:
                return True
        return False
    except:
        return False

In [6]:
anony_lst = []
gpt_lst = []
anony_flag = False
gpt_flag = False
for i in tqdm(range(len(clg))):
    anony_flag = False
    try:
        for j in clg['conversation'].iloc[i].split('{'):
            if pure_en(j):
                if ('Anonymous' in j[:20]):
                    anony_flag = True
                    anony_temp = j[31:-3].strip().strip('"').strip("'").strip('?')
                if ('Chat GPT' in j[0:20]):
                    gpt_flag = True
                    gpt_temp = j[31:-3].strip().strip('"').strip("'").strip('?')
                if anony_flag and gpt_flag:
                    anony_lst.append(anony_temp)
                    gpt_lst.append(gpt_temp)
                    anony_flag = False
                    gpt_flag = False
        anony_flag = False
        gpt_flag = False
    except:
        pass
        anony_flag = False
        gpt_flag = False

100%|██████████| 89311/89311 [1:41:31<00:00, 14.66it/s]  


In [ ]:
print(len(anony_lst))
print(len(gpt_lst))

383589
383589


In [19]:
joblib.dump(anony_lst, r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\anony_lst_all.joblib')
joblib.dump(gpt_lst, r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\gpt_lst_all.joblib')

['D:\\ML_Projects\\AI_Tech_ChatBot\\Data\\ChatGPT_chatlogs\\gpt_lst_all.joblib']

In [8]:
gpt_data = pd.DataFrame({'Question':anony_lst, 'Answer':gpt_lst})

In [10]:
gpt_data.drop(gpt_data[(gpt_data['Question']=='') | (gpt_data['Answer']=='')].index, inplace=True)

In [11]:
len(gpt_data)

383234

In [12]:
gpt_data.to_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_all.csv', index=False)

In [13]:
vocab = ' '.join(anony_lst + gpt_lst)
vocab = sorted(set(vocab.split()))

In [14]:
len(vocab)

1035821

In [17]:
joblib.dump(vocab, r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_Vocab_all.joblib')

['D:\\ML_Projects\\AI_Tech_ChatBot\\Data\\ChatGPT_chatlogs\\GPT_Vocab_all.joblib']

In [ ]:
np.random.choice(vocab, 100, replace=False)

In [20]:
anony_lst

['What is contrastive loss',
 'Do we need labels for contrastive learning',
 'Right, but we still need to know the difference between cats and dogs to sample positive and negative data',
 'What is the best approach for sampling in contrastive learning',
 'What are the differences and pros/cons of triplet loss compared to contrastive loss',
 'write a poem about John Kaercher, a corporate lawyer who cannot stop eating expired cottage cheese during client meetings',
 'What is contrastive loss',
 'Do we need labels for contrastive learning',
 'Right, but we still need to know the difference between cats and dogs to sample positive and negative data',
 'What is the best approach for sampling in contrastive learning',
 'What are the differences and pros/cons of triplet loss compared to contrastive loss',
 'What kind of contrastive learning is used in the CLIP paper',
 'Is instance discrimination a more difficult learning objective that standard contrastive learning (distinguishing between di

In [24]:
import re
re.sub(r'^[^a-zA-Z0-9]+|[^a-zA-Z0-9]+$', '', "#1 / 1        You are a producer at Gameloft. Give at least 3 ideas to solve the problem.")

'1 / 1        You are a producer at Gameloft. Give at least 3 ideas to solve the problem'

In [33]:
vocab = joblib.load(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_Vocab_all.joblib')

In [35]:
for i in range(20):
    print(vocab[np.random.randint(0, len(vocab))])

starcast
Nationale
audiences."
pulpit,
biodegradation.
pneumothorax.
00:00:02
reverence
What\'
-version"
İnanç
lather,
KEY([PreviousResidenceReturnReasonID])
SuperCollider,
LIB2
conjectural.
https://www.cherokee.org/chr/%E1%8F%A3%E1%8E%B3%E1%8E%A9-%E1%8E%A0%E1%8F%B0%E1%8E%B5%E1%8E%A4%E1%8F%99%E1%8F%A2%E1%8F%92/maps/
Halit
experienced)
Hermeneutics,


In [3]:
gpt_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_all.csv')

In [8]:
print_lim = 0
for i in range(len(gpt_data)):
    words = gpt_data['Question'][i].split()
    vecs = []
    for w in words:
        vecs.append(w)
    print(vecs)
    vecs = []
    if print_lim == 10:
        break
    print_lim+=1

['What', 'is', 'contrastive', 'loss']
['Do', 'we', 'need', 'labels', 'for', 'contrastive', 'learning']
['Right,', 'but', 'we', 'still', 'need', 'to', 'know', 'the', 'difference', 'between', 'cats', 'and', 'dogs', 'to', 'sample', 'positive', 'and', 'negative', 'data']
['What', 'is', 'the', 'best', 'approach', 'for', 'sampling', 'in', 'contrastive', 'learning']
['What', 'are', 'the', 'differences', 'and', 'pros/cons', 'of', 'triplet', 'loss', 'compared', 'to', 'contrastive', 'loss']
['write', 'a', 'poem', 'about', 'John', 'Kaercher,', 'a', 'corporate', 'lawyer', 'who', 'cannot', 'stop', 'eating', 'expired', 'cottage', 'cheese', 'during', 'client', 'meetings']
['What', 'is', 'contrastive', 'loss']
['Do', 'we', 'need', 'labels', 'for', 'contrastive', 'learning']
['Right,', 'but', 'we', 'still', 'need', 'to', 'know', 'the', 'difference', 'between', 'cats', 'and', 'dogs', 'to', 'sample', 'positive', 'and', 'negative', 'data']
['What', 'is', 'the', 'best', 'approach', 'for', 'sampling', 'in',

In [1]:
import pandas as pd

In [3]:
gpt_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_all.csv')

In [5]:
for i in range(len(gpt_data)):
    try:
        if (len(gpt_data['Question'][i].split()) < 4) or (len(gpt_data['Answer'][i].split()) < 4) or (len(gpt_data['Question'][i].split()) > 100) or (len(gpt_data['Answer'][i].split()) > 100):
            gpt_data.iloc[i-1].drop(inplace=True)
    except:
        gpt_data.iloc[i-1].drop(inplace=True)

,Question,Answer
0,What is contrastive loss,Contrastive loss is a type of loss function th...
1,Do we need labels for contrastive learning,"In contrastive learning, it is not necessary t..."
2,"Right, but we still need to know the differenc...","Yes, that's correct. In order to use contrasti..."
3,What is the best approach for sampling in cont...,The best approach for sampling in contrastive ...
4,What are the differences and pros/cons of trip...,Triplet loss and contrastive loss are both typ...


In [11]:
gpt_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_all.csv')
mask = (gpt_data['Question'].str.split().str.len().between(4, 100)) & (gpt_data['Answer'].str.split().str.len().between(4, 50))
gpt_data = gpt_data[mask].reset_index(drop=True)
gpt_data.to_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_all_filtered_1.csv', index=False)

In [10]:
gpt_data = pd.read_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_all.csv')
def is_word_count_valid(text):
    try:
        word_count = len(str(text).split())
        return 4 <= word_count <= 100
    except Exception as e:
        return False
valid_questions = gpt_data['Question'].apply(is_word_count_valid)
valid_answers = gpt_data['Answer'].apply(is_word_count_valid)
valid_rows = valid_questions & valid_answers
filtered_gpt_data = gpt_data[valid_rows].reset_index(drop=True)
gpt_data.to_csv(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\ChatGPT_chatlogs\GPT_chatlogs_all_filtered_1.csv', index=False)

In [ ]:
from datasets import load_dataset
ds = load_dataset("Isotonic/human_assistant_conversation")

In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/bitext/Bitext-customer-support-llm-chatbot-training-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")

df.to_excel(r'D:\ML_Projects\LLM-From-Scratch-For-ChatBots-GPT2\Data\Huggingface\Customer_Support_Hugg.xlsx')

d:\VENVs\chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
import re

text = """
How to change the item that is defective? [SEP]  goal is to assist you in modifying the item in your order with the number {{Order Number}}.
We understand the importance of making this an necessary adjustments to your order to your desired location. To proceed with the modifying your order, could you please provide more details on the item(s) you would like to change? This will help us provide you with accurate and tailored assistance.!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
"""

pattern = r'\[SEP\]\s*(.*)'
match = re.search(pattern, text, re.DOTALL)
if match:
    result = match.group(1)
    result = re.sub(r'!{3,}', '', result)
    print(result)
else:
    print(text)

goal is to assist you in modifying the item in your order with the number {{Order Number}}.
We understand the importance of making this an necessary adjustments to your order to your desired location. To proceed with the modifying your order, could you please provide more details on the item(s) you would like to change? This will help us provide you with accurate and tailored assistance.

